In [1]:
import json
import numpy as np
import pandas as pd
import ipywidgets as widgets
from matplotlib import pyplot as plt
from matplotlib import colors as colors

import scipy.stats
from scipy import stats

from datetime import datetime
from datetime import timedelta

from bqplot import OrdinalScale, DateScale, LinearScale, Lines, Axis, Figure

from ipyleaflet import Map, GeoJSON, Marker
from ipywidgets import interact

In [2]:
# Нью-Йорк вписан в прямоугольник от -74.25559 до -73.70001 градусов долготы и от 40.49612 до 40.91553 широты. 
NY = (-74.25559, -73.70001, 40.49612, 40.91553)

In [3]:
ny_west = NY[0]
ny_east = NY[1]
ny_south = NY[2]
ny_north = NY[3]

Функция загрузки, фильтрации и агрегации данных

In [4]:
def aggregate_data(fname, dataname):
    # Загрузка данных
    data = pd.read_csv(fname, header = 0, sep=',')

    # ФИЛЬТРАЦИЯ
    # Удаление поездок с нулевой длительностью
    data.drop(data[data['tpep_pickup_datetime'] == data['tpep_dropoff_datetime']].index, axis=0, inplace=True)
    data.drop(data[data['tpep_pickup_datetime'] > data['tpep_dropoff_datetime']].index, axis=0, inplace=True)

    # Удаление поездок с нулевым количеством пассажиров
    data.drop(data[data['passenger_count'] == 0].index, axis=0, inplace=True)

    # Удаление поездок с нулевым расстоянием поездки по счётчику
    data.drop(data[data['trip_distance'] == 0].index, axis=0, inplace=True)

    # Удаление поездок с координатами начала, не попадающими в прямоугольник Нью-Йорка
    data.drop(data[(data['pickup_longitude'] < -74.25559) | (data['pickup_longitude']  > -73.70001) | (data['pickup_latitude']  < 40.49612) | (data['pickup_latitude']  > 40.91553)].index, axis=0, inplace=True)

    # Новый столбец date, показывает час начала поездки
    data.insert(1, 'timestamp', data.tpep_pickup_datetime.apply(lambda x : x.split(':')[0]))

    X = data.pickup_longitude
    Y = data.pickup_latitude
    bin_count = 50
    bin_stat = stats.binned_statistic_2d(X, Y, None, statistic='count',
                                         bins=bin_count,
                                         range=[[NY[0], NY[1]], [NY[2], NY[3]]],
                                         expand_binnumbers=True)
    regions = (bin_stat.binnumber[0] - 1) * bin_count + bin_stat.binnumber[1]

    # Добавим еще один столбец - регион начала поездки
    data.insert(1, 'region_id', regions)

    # Теперь собираем аггрегированную таблицу из трех столбцов: дата, час, номер ячейки, 
    # количество поездок из этой ячейки в этот час в эту дату
    data_agg = data[['timestamp','region_id']].groupby(['region_id', 'timestamp']).size().reset_index(name='actual')
    
    # Сохраняем
    data_agg.to_csv(dataname, sep = ',', index = False, mode = 'a')

In [5]:
'''%%time

aggregate_data('yellow_tripdata_2016-05.csv','data_agg_w7.csv')
aggregate_data('yellow_tripdata_2016-06.csv','data_agg_w7.csv')
'''

Wall time: 4min 42s


In [6]:
data_agg_full = pd.read_csv('data_agg_w7.csv', header = 0, sep=',')

# Удаляем лишние заголовки
data_agg_full.drop(data_agg_full.loc[data_agg_full['region_id'] == 'region_id'].index, axis = 0, inplace = True)

# Приводим к необходимому типу данных
data_agg_full['actual'] = pd.to_numeric(data_agg_full['actual'])
data_agg_full['timestamp'] = pd.to_datetime(data_agg_full['timestamp'])
data_agg_full['region_id'] = pd.to_numeric(data_agg_full['region_id'])
data_agg_full['date'] = data_agg_full['timestamp'].dt.date 
data_agg_full['day'] = [data_agg_full['timestamp'][i].day for i in data_agg_full.index]
data_agg_full['hour'] = [data_agg_full['timestamp'][i].hour for i in data_agg_full.index]

rides_total = data_agg_full.loc[(data_agg_full.timestamp > '2016-04-30 23:00:00') & (data_agg_full.timestamp < '2016-06-01 00:00:00')][['region_id','actual']].groupby(['region_id']).sum()
rides_total['mean'] = rides_total['actual']/31./24.
index2select = np.array(rides_total.loc[(rides_total['mean'] >= 5)].index)

data_agg = data_agg_full.loc[data_agg_full.region_id.isin(index2select)]

print data_agg.shape
data_agg.head()

C:\ProgramData\Anaconda2\lib\site-packages\IPython\core\interactiveshell.py:2718: DtypeWarning: Columns (0,2) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


(145492, 6)


,region_id,timestamp,actual,date,day,hour
3207,1075,2016-05-01 00:00:00,71,2016-05-01,1,0
3208,1075,2016-05-01 01:00:00,37,2016-05-01,1,1
3209,1075,2016-05-01 02:00:00,10,2016-05-01,1,2
3210,1075,2016-05-01 03:00:00,14,2016-05-01,1,3
3211,1075,2016-05-01 04:00:00,6,2016-05-01,1,4


Прогноз на июнь 2016 года

In [7]:
submission = pd.read_csv('2016_06_predictions_with_additional_features.csv',sep=',',index_col=0,header=0)
submission.head()

,y
id,
1075_2016-05-31_23_1,24.196970
1075_2016-05-31_23_2,12.409091
1075_2016-05-31_23_3,6.512821
1075_2016-05-31_23_4,4.290909
1075_2016-05-31_23_5,5.604396


In [8]:
# Преобразуем индексы в столбцы таблицы
submission = submission.reset_index(level=['id'])
submission.head()

,id,y
0,1075_2016-05-31_23_1,24.196970
1,1075_2016-05-31_23_2,12.409091
2,1075_2016-05-31_23_3,6.512821
3,1075_2016-05-31_23_4,4.290909
4,1075_2016-05-31_23_5,5.604396


In [9]:
# Разбираем id на составляющие, а именно выделяем номер зоны, дату, час и шаг прогноза
submission['region'] = [submission.id[i][:4] for i in range(len(submission.id))]
submission['date'] = [submission.id[i][5:15] for i in range(len(submission.id))]
submission['region'] = pd.to_numeric(submission['region'])
submission['date'] = pd.to_datetime(submission['date'])

submission['hour'] = [submission.id[i][16:18].replace("_", "") for i in range(len(submission.id))]
submission['number_predict'] = [submission.id[i][len(submission.id[i])-1:] for i in range(len(submission.id))]
submission['hour'] = pd.to_numeric(submission['hour'])
submission['number_predict'] = pd.to_numeric(submission['number_predict'])

submission['timestamp'] = [datetime(submission['date'][i].year, submission['date'][i].month, submission['date'][i].day, submission['hour'][i]) for i in range(len(submission.id))]
submission['timestamp'] = pd.to_datetime(submission['timestamp'])

submission.drop(['id','date','hour'], axis = 1, inplace = True) # удаляем вспомогательные поля

submission.head()

,y,region,number_predict,timestamp
0,24.196970,1075,1,2016-05-31 23:00:00
1,12.409091,1075,2,2016-05-31 23:00:00
2,6.512821,1075,3,2016-05-31 23:00:00
3,4.290909,1075,4,2016-05-31 23:00:00
4,5.604396,1075,5,2016-05-31 23:00:00


Преобразуем представленный формат данных с тем, чтобы объединить факт и прогноз на июнь 2016 года

In [10]:
d1 = submission.loc[submission['number_predict'] == 1]
d1['forecast_1'] = d1.y
d1['timestamp'] = d1.loc[d1['number_predict'] == 1]['timestamp'] + timedelta(hours=1)
d1.drop(['y'], axis = 1, inplace=True)

d2 = submission.loc[submission['number_predict'] == 2]
d2['forecast_2'] = d2.y
d2['timestamp'] = d2.loc[d2['number_predict'] == 2]['timestamp'] + timedelta(hours=2)
d2.drop(['y'], axis = 1, inplace=True)

d3 = submission.loc[submission['number_predict'] == 3]
d3['forecast_3'] = d3.y
d3['timestamp'] = d3.loc[d3['number_predict'] == 3]['timestamp'] + timedelta(hours=3)
d3.drop(['y'], axis = 1, inplace=True)

d4 = submission.loc[submission['number_predict'] == 4]
d4['forecast_4'] = d4.y
d4['timestamp'] = d4.loc[d4['number_predict'] == 4]['timestamp'] + timedelta(hours=4)
d4.drop(['y'], axis = 1, inplace=True)

d5 = submission.loc[submission['number_predict'] == 5]
d5['forecast_5'] = d5.y
d5['timestamp'] = d5.loc[d5['number_predict'] == 5]['timestamp'] + timedelta(hours=5)
d5.drop(['y'], axis = 1, inplace=True)

d6 = submission.loc[submission['number_predict'] == 6]
d6['forecast_6'] = d6.y
d6['timestamp'] = d6.loc[d6['number_predict'] == 6]['timestamp'] + timedelta(hours=6)
d6.drop(['y'], axis = 1, inplace=True)

C:\ProgramData\Anaconda2\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
C:\ProgramData\Anaconda2\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until
C:\ProgramData\Anaconda2\lib\site-packages\ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/st

In [11]:
data = pd.merge(data_agg[['region_id', 'timestamp','actual']].loc[(data_agg.timestamp >= '2016-06-01 00:00:00') & (data_agg.timestamp <= '2016-06-30 23:00:00')], d1, left_on = ['region_id', 'timestamp'], right_on = ['region', 'timestamp'], how='left').fillna(0)
data.drop(['region'], axis = 1, inplace = True)

In [12]:
data = pd.merge(data, d2[['region', 'timestamp','forecast_2']], left_on = ['region_id', 'timestamp'], right_on = ['region', 'timestamp'], how='left').fillna(0)
data.drop(['region'], axis = 1, inplace = True)

In [13]:
data = pd.merge(data, d3[['region', 'timestamp','forecast_3']], left_on = ['region_id', 'timestamp'], right_on = ['region', 'timestamp'], how='left').fillna(0)
data.drop(['region'], axis = 1, inplace = True)

In [14]:
data = pd.merge(data, d4[['region', 'timestamp','forecast_4']], left_on = ['region_id', 'timestamp'], right_on = ['region', 'timestamp'], how='left').fillna(0)
data.drop(['region'], axis = 1, inplace = True)

In [15]:
data = pd.merge(data, d5[['region', 'timestamp','forecast_5']], left_on = ['region_id', 'timestamp'], right_on = ['region', 'timestamp'], how='left').fillna(0)
data.drop(['region'], axis = 1, inplace = True)

In [16]:
data = pd.merge(data, d6[['region', 'timestamp','forecast_6']], left_on = ['region_id', 'timestamp'], right_on = ['region', 'timestamp'], how='left').fillna(0)
data.drop(['region'], axis = 1, inplace = True)

In [17]:
data.rename(columns={'region_id':'region'}, inplace=True)

Таким образом, мы получаем фактические и прогнозные данные в одном dataframe

In [18]:
print data.shape
data.head()

(71511, 10)


,region,timestamp,actual,number_predict,forecast_1,forecast_2,forecast_3,forecast_4,forecast_5,forecast_6
0,1075,2016-06-01 00:00:00,26,1.0,24.196970,0.000000,0.000000,0.000000,0.000000,0.0
1,1075,2016-06-01 01:00:00,14,1.0,12.742424,12.409091,0.000000,0.000000,0.000000,0.0
2,1075,2016-06-01 02:00:00,5,1.0,6.196970,6.409091,6.512821,0.000000,0.000000,0.0
3,1075,2016-06-01 03:00:00,2,1.0,3.242424,3.136364,5.551282,4.290909,0.000000,0.0
4,1075,2016-06-01 04:00:00,1,1.0,3.000000,4.000000,4.717949,4.927273,5.604396,0.0


In [19]:
# Фактические значения
data_real = data[['region','timestamp','actual']].pivot(index='timestamp', columns='region', values='actual')
data_real.fillna(0,inplace=True)
print data_real.shape
data_real.head()

(720, 102)


region,1075,1076,1077,1125,1126,1127,1128,1129,1130,1131,...,1630,1684,1733,1734,1783,2068,2069,2118,2119,2168
timestamp,,,,,,,,,,,,,,,,,,,,,
2016-06-01 00:00:00,26.0,30.0,19.0,39.0,71.0,163.0,181.0,219.0,326.0,56.0,...,1.0,1.0,6.0,262.0,107.0,114.0,7.0,169.0,6.0,85.0
2016-06-01 01:00:00,14.0,21.0,6.0,26.0,49.0,101.0,136.0,144.0,252.0,27.0,...,5.0,0.0,0.0,137.0,1.0,27.0,16.0,105.0,4.0,42.0
2016-06-01 02:00:00,5.0,25.0,4.0,18.0,25.0,42.0,74.0,123.0,245.0,30.0,...,8.0,0.0,1.0,25.0,0.0,11.0,0.0,31.0,0.0,10.0
2016-06-01 03:00:00,2.0,5.0,3.0,4.0,16.0,36.0,36.0,85.0,241.0,24.0,...,7.0,0.0,1.0,44.0,0.0,13.0,4.0,51.0,1.0,0.0
2016-06-01 04:00:00,1.0,8.0,3.0,6.0,26.0,27.0,46.0,54.0,129.0,11.0,...,4.0,0.0,0.0,0.0,2.0,23.0,3.0,33.0,3.0,1.0


In [20]:
dict_real = data_real.to_dict('list')

In [21]:
# Прогнозные значения
data_predict1 =  data[['region','timestamp','forecast_1']].pivot(index='timestamp', columns='region', values='forecast_1')
data_predict1.fillna(0,inplace=True)
dict_predict1 = data_predict1.to_dict('list')

In [22]:
data_predict2 = data[['region','timestamp','forecast_2']].pivot(index='timestamp', columns='region', values='forecast_2')
data_predict2.fillna(0,inplace=True)
dict_predict2 = data_predict2.to_dict('list')

In [23]:
data_predict3 = data[['region','timestamp','forecast_3']].pivot(index='timestamp', columns='region', values='forecast_3')
data_predict3.fillna(0,inplace=True)
dict_predict3 = data_predict3.to_dict('list')

In [24]:
data_predict4 = data[['region','timestamp','forecast_4']].pivot(index='timestamp', columns='region', values='forecast_4')
data_predict4.fillna(0,inplace=True)
dict_predict4 = data_predict4.to_dict('list')

In [25]:
data_predict5 = data[['region','timestamp','forecast_5']].pivot(index='timestamp', columns='region', values='forecast_5')
data_predict5.fillna(0,inplace=True)
dict_predict5 = data_predict5.to_dict('list')

In [26]:
data_predict6 = data[['region','timestamp','forecast_6']].pivot(index='timestamp', columns='region', values='forecast_6')
data_predict6.fillna(0,inplace=True)
dict_predict6 = data_predict6.to_dict('list')

In [27]:
dict_predict = {1 : dict_predict1,
               2: dict_predict2,
               3: dict_predict3,
               4: dict_predict4,
               5: dict_predict5,
               6: dict_predict6}

Список данных готов

In [28]:
tripdata = {'actual': dict_real,
           'forecast': dict_predict,
           'hours': [str(data.groupby(data.timestamp).count().index[i])[:-6] for i in range(len(data.groupby(data.timestamp).count().index))]} 

In [29]:
class DateRangePicker(object):
    def __init__(self, start, end, freq='H', fmt='%Y-%m-%d %H', cb=None):
        """
        Parameters
        ----------
        start : string or datetime-like
            Left bound of the period
        end : string or datetime-like
            Right bound of the period
        freq : string or pandas.DateOffset, default='H'
            Frequency strings can have multiples, e.g. '5H' 
        fmt : string, default = '%Y-%m-%d %H'
            Format to use to display the selected period

        """
        date_range = pd.date_range(start=start, end=end, freq=freq)
        options = [(item.strftime(fmt), np.datetime64(item)) for item in date_range]
        self.slider_start = widgets.SelectionSlider(
            description='Start:',
            options=options,
            continuous_update=False,
            layout=widgets.Layout(width='100%')
        )
        self.slider_end = widgets.SelectionSlider(
            description='End:',
            options=options,
            continuous_update=False,
            value=options[-1][1],
            layout=widgets.Layout(width='100%')
        )
        self.cb = cb
        self.slider_start.observe(self.handle_change, 'value')
        self.slider_end.observe(self.handle_change, 'value')
        self.widget = widgets.Box(children=[self.slider_start, self.slider_end])     
    
    def handle_change(self, d):
        if self.cb:
            self.cb(self.slider_start.value, self.slider_end.value)

In [30]:
def get_coords(region):
    i = int((region - 1) / 50)
    j = (region - 1) % 50
    dx = (ny_east - ny_west) / 50
    dy = (ny_north - ny_south) / 50
    west = i * dx + ny_west
    south = j * dy + ny_south
    return west, west + dx, south, south + dy

In [31]:
regions = sorted(tripdata['actual'].keys())
steps = sorted(tripdata['forecast'].keys())

In [32]:
hours = np.array(map(np.datetime64, tripdata['hours']))
for region in regions:
    tripdata['actual'][region] = np.array(tripdata['actual'][region])
    for step in steps:
        tripdata['forecast'][step][region] = np.array(tripdata['forecast'][step][region])

Основной график - содержит фактические и прогнозируемые данные о количестве поездок

In [33]:
x_sc = DateScale()
y_sc = LinearScale()

selected = {'step': steps[0], 'region': regions[0], 
            'start': hours[0], 'end': hours[-1]}

aline = Lines(scales={'x': x_sc, 'y': y_sc},
             stroke_width=1, colors=['blue'], display_legend=True, 
             labels=[u'Actual']
            )
fline = Lines(scales={'x': x_sc, 'y': y_sc},
             stroke_width=1, colors=['red'], display_legend=True, 
             labels=[u'Forecast']
            )

ax_x = Axis(scale=x_sc, grid_lines='solid', label=u'Hour')
ax_y = Axis(scale=y_sc, orientation='vertical', tick_format='0.0f',
            grid_lines='solid', label=u'Trip count')

f = Figure(marks=[aline, fline], axes=[ax_x, ax_y], min_width=1000, min_height=400, 
           title=u'New York yellow taxi trips', legend_location='top-right', layout=widgets.Layout(width='50%'),)

def update_lines():
    v = (hours >= selected['start']) & (hours <= selected['end'])
    x = hours[v]
    aline.x = x
    fline.x = x
    fline.y = tripdata['forecast'][selected['step']][selected['region']][v]
    aline.y = tripdata['actual'][selected['region']][v]
    
update_lines()

Центр карты поставим в середину наших регионов

In [34]:
all_coords = map(get_coords, map(int, regions))
min_lon = min(map(lambda x: x[0], all_coords))
max_lon = max(map(lambda x: x[1], all_coords))
min_lat = min(map(lambda x: x[2], all_coords))
max_lat = max(map(lambda x: x[3], all_coords))

На карту нанесём наши регионы, окрашенные по количеству среднего количества поездок

In [35]:
def make_geo_json(regions):
    reg_avg = {region: np.mean(tripdata['actual'][region]) for region in regions}
    reg_max = max(reg_avg.values())
    colormap = plt.get_cmap('Blues')
    data = {'type': 'FeatureCollection', 'features': []}
    for region in regions:
        i = int(float(colormap.N) * np.log(reg_avg[region]) / np.log(reg_max))
        rgb = colormap(i)[:3] # will return rgba, we take only first 3 so we get rgb
        color = colors.rgb2hex(rgb)
        w, e, s, n = get_coords(int(region))
        coordinates = [[w, s], [w, n], [e, n], [e, s]]
        data['features'].append({
            'id': region,
            'type': 'Feature',
            'geometry': {'coordinates': [[coordinates]], 'type': 'MultiPolygon'},
            'properties': {'style': {'color': color, 'fillColor': color, 
                                     'fillOpacity': 0.5, 'weight': 1}},
        })
    return GeoJSON(data=data, hover_style={'fillColor': 'red'})

In [36]:
# Загружаем регионы
data_regions = pd.read_csv('regions.csv', header=0, sep=';')
data_regions.head()

,region,west,east,south,north
0,1,-74.25559,-74.244478,40.496120,40.504508
1,2,-74.25559,-74.244478,40.504508,40.512896
2,3,-74.25559,-74.244478,40.512896,40.521285
3,4,-74.25559,-74.244478,40.521285,40.529673
4,5,-74.25559,-74.244478,40.529673,40.538061


In [37]:
from ipywidgets import HTML
from ipyleaflet import Map, Marker, Popup

center = [(min_lat + max_lat) / 2, (min_lon + max_lon) / 2]
zoom = 11

m = Map(center=center, zoom=zoom, scroll_wheel_zoom=True, 
        layout=widgets.Layout(width='50%', height='450px'))


def region_handler(event=None, id=None, properties=None):
    selected['region'] = id
    update_lines()
    
def region_popup(event=None, id=None, properties=None):
    selected['region'] = id
    message = HTML()
    message.value = "Region: " + str(id) + "<br>"\
    + "Time Shift: " + str(selected['step']) + "<br>"\
    + "Actual: " + str(tripdata['actual'][id][len(tripdata['actual'][id])-7+selected['step']]) + "<br>"\
    + "Forecast: " + str(round(tripdata['forecast'][selected['step']][id][len(tripdata['forecast'][selected['step']][id])-7+selected['step']],2)) + "<br>"\
    + "Residual: " + str(round((tripdata['actual'][id][len(tripdata['actual'][id])-7+selected['step']] - tripdata['forecast'][selected['step']][id][len(tripdata['forecast'][selected['step']][id])-7+selected['step']])/tripdata['forecast'][selected['step']][id][len(tripdata['forecast'][selected['step']][id])-7+selected['step']]*100,2)) + " %"
    popup = Popup(
    location=[float((data_regions[data_regions.region==id]['south']) + float(data_regions[data_regions.region==id]['north']))/2., (float(data_regions[data_regions.region==id]['west']) + float(data_regions[data_regions.region==id]['east']))/2.],
    child=message,  
    close_button=True,
    auto_close=False
    )
    m.add_layer(popup)

Объект для выбора начала и конца рассматриваемого периода

In [38]:
def date_range_handler(start, end):
    selected['start'] = start
    selected['end'] = end
    update_lines()

Объект для выбора шага прогноза (мы сделали 6 моделей для прогнозирования на 1, ..., 6 часов)

In [39]:
def step_pair(step):
    label = 'hour' if step == 1 else 'hours'
    return (str(step) + ' ' + label, step)

def step_handler(d):
    selected['step'] = d['new']
    update_lines()

In [40]:
# Виджет карта + график факт/прогноз
map_and_time_series = widgets.HBox([m, f])

In [42]:
# Дата начала и дата окончания
drp = DateRangePicker(start=hours[0], end=hours[-1], cb=date_range_handler)
drp.widget.layout = widgets.Layout(width='100%', margin='50px 0 0 0')

# Шаг прогноза
options = map(step_pair, steps)
ss = widgets.SelectionSlider(description=u'Step:', options=options, continuous_update=False, 
                             value=options[0][1], layout=widgets.Layout(width='48.5%', margin='20px 0 0 0'))
ss.observe(step_handler, 'value')

# График фактического и прогнозируемого спроса на такси
f = Figure(marks=[aline, fline], axes=[ax_x, ax_y], min_width=1000, min_height=400, 
           title=u'New York yellow taxi trips', legend_location='top-right', layout=widgets.Layout(width='50%'),)

# Карта 
m = Map(center=center, zoom=zoom, scroll_wheel_zoom=True, 
        layout=widgets.Layout(width='50%', height='450px'))

# Сетка регионов
g = make_geo_json(regions)
g.on_hover(region_handler)
g.on_click(region_popup) # Вызов popup по щелчку мыши
m.add_layer(g)

map_and_time_series = widgets.HBox([m, f])
Demo = widgets.VBox([drp.widget, ss, map_and_time_series])
Demo

A Jupyter Widget